# Climate Data Cleaning Notebook

In [ ]:
!pip install fancyimpute -q

     |████████████████████████████████| 154 kB 7.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns 

# Plot missing values
import missingno as msno

# Model
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from numpy import nan as NA

#imports imputers
from sklearn import preprocessing
from fancyimpute import KNN
from fancyimpute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

## Climate


## Try and make weather data seasonal

In [ ]:
def assign_season(month):
  if month in ['Dec', 'Jan', 'Feb']:
    return 'Winter'
  elif month in ['Mar', 'Apr', 'May']:
    return 'Spring'
  elif month in ['Jun', 'Jul', 'Aug']:
    return 'Summer'
  elif month in ['Sep', 'Oct', 'Nov']:
    return 'Fall'


In [ ]:
#make columns lower case
weather_final.columns = weather_final.columns.str.lower()
#apply seasonality lable to the weather data
weather_final['season'] = weather_final['month'].apply(assign_season)

In [ ]:
seasons = weather_final.groupby(['year', 'season']).mean()
#take max of max temperature and min of min temperature and then calulcate difference
seasons['highest_temperature'] = weather_final.groupby(['year', 'season'])['highest_temperature'].max()
seasons['lowest_temperature'] = weather_final.groupby(['year', 'season'])['lowest_temperature'].min()
seasons['temperature_difference'] = seasons['highest_temperature'] - seasons['lowest_temperature']
#take sum of snowfall and precipitation
seasons['precipitation_(inches)'] = weather_final.groupby(['year', 'season'])['precipitation_(inches)'].sum()
seasons['snowfall_(inches)'] = weather_final.groupby(['year', 'season'])['snowfall_(inches)'].sum()

In [ ]:
seasons

average_temperature  highest_temperature  lowest_temperature  \
year season                                                                 
1891 Fall              44.450683            87.000000           25.811499   
     Spring            37.145349            71.898202           25.811499   
     Summer            49.926996           100.000000           25.811499   
     Winter            35.169842            71.898202           14.000000   
1892 Fall              45.376329            86.000000           25.811499   
...                          ...                  ...                 ...   
2021 Winter            30.149028            76.343646           -9.000000   
2022 Fall              49.393101           102.000000           25.811499   
     Spring            44.859956            81.515844           12.000000   
     Summer            63.798530           112.000000           25.811499   
     Winter            26.383561            79.549667          -13.000000   

             precipitation_(inches)  temperature_difference  snowfall_(inches)  
year season                                                                     
1891 Fall                  1.457877               61.188501                0.0  
     Spring                2.723630               46.086703                1.0  
     Summer                1.355291               74.188501                0.0  
     Winter                4.105754               57.898202               38.8  
1892 Fall                  1.757877               60.188501                0.0  
...                             ...                     ...                ...  
2021 Winter               34.633630               85.343646              273.4  
2022 Fall                 10.188768               76.188501                0.0  
     Spring               26.235754               69.515844                3.6  
     Summer               22.743382               86.188501                0.0  
     Winter               12.800000               92.549667              108.8  

[528 rows x 6 columns]

In [ ]:
seasons.loc[2021]

,average_temperature,highest_temperature,lowest_temperature,precipitation_(inches),temperature_difference,snowfall_(inches)
season,,,,,,
Fall,44.069972,94.000000,13.443607,36.183630,80.556393,7.5
Spring,43.514632,96.000000,15.000000,5.573630,81.000000,10.8
Summer,62.358860,118.000000,17.245319,9.221185,100.754681,0.0
Winter,30.149028,76.343646,-9.000000,34.633630,85.343646,273.4


In [ ]:
print(weather_final[weather_final['season'] == 'Fall'].shape)
print(weather_final[weather_final['season'] == 'Winter'].shape)
print(weather_final[weather_final['season'] == 'Spring'].shape)
print(weather_final[weather_final['season'] == 'Summer'].shape)

(2913, 10)
(2913, 10)
(2922, 10)
(2922, 10)


In [ ]:
#download to github
from google.colab import drive
drive.mount('drive')

#avg.to_csv('average_temp.csv')
#!cp average_temp.csv "drive/My Drive/"

seasons.to_csv('seasons.csv')
!cp seasons.csv "drive/My Drive/"

Mounted at drive


# Prism Data

In [ ]:
w = pd.read_csv('https://raw.githubusercontent.com/mckenzie-maidl/data5100-climate-agriculture/main/Raw%20Data/PRISM/WallaWalla.csv', delimiter=',')
w

,Date,ppt (inches),tmin (degrees F),tmean (degrees F),tmax (degrees F),tdmean (degrees F),vpdmin (hPa),vpdmax (hPa),County
0,1895-01,2.55,23.2,29.9,36.6,23.4,0.47,2.64,Walla Walla
1,1895-02,0.58,32.4,40.3,48.2,28.7,0.97,5.86,Walla Walla
2,1895-03,0.81,33.6,44.0,54.5,28.3,1.32,9.14,Walla Walla
3,1895-04,0.58,40.7,54.0,67.4,32.4,2.63,16.43,Walla Walla
4,1895-05,1.60,45.4,57.9,70.4,37.4,2.77,18.02,Walla Walla
...,...,...,...,...,...,...,...,...,...
1519,2021-08,0.06,58.3,72.7,87.1,48.5,5.21,34.30,Walla Walla
1520,2021-09,0.61,50.4,63.8,77.2,43.5,2.98,23.36,Walla Walla
1521,2021-10,1.50,41.6,52.7,63.8,40.1,1.25,12.40,Walla Walla
1522,2021-11,2.16,36.9,44.2,51.4,38.1,0.50,5.10,Walla Walla


In [ ]:
#Prism
file_names = ['Benton.csv', 'Chelan.csv', 'Douglas.csv', 'Franklin.csv', 'Grant.csv', 'Klickitat.csv', 'Okanogan.csv', 'WallaWalla.csv','Yakima.csv']

prism = pd.read_csv('https://raw.githubusercontent.com/mckenzie-maidl/data5100-climate-agriculture/main/Raw%20Data/PRISM/Adams.csv')

for name in file_names:
  f = pd.read_csv('https://raw.githubusercontent.com/mckenzie-maidl/data5100-climate-agriculture/main/Raw%20Data/PRISM/' + name, delimiter=',')
  prism = pd.concat([prism, f])

In [ ]:
prism

,Date,ppt (inches),tmin (degrees F),tmean (degrees F),tmax (degrees F),tdmean (degrees F),vpdmin (hPa),vpdmax (hPa),County
0,1895-01,2.00,20.1,26.7,33.2,21.2,0.45,1.93,Adams
1,1895-02,0.38,28.4,36.3,44.2,25.7,0.78,4.76,Adams
2,1895-03,0.38,28.5,40.0,51.5,26.4,0.90,7.99,Adams
3,1895-04,0.41,36.3,50.8,65.3,31.3,1.68,15.49,Adams
4,1895-05,0.91,41.6,55.3,69.0,37.0,1.78,17.02,Adams
...,...,...,...,...,...,...,...,...,...
1519,2021-08,0.02,56.6,71.8,87.1,49.2,4.61,33.66,Yakima
1520,2021-09,0.52,46.7,61.6,76.5,43.7,2.04,22.51,Yakima
1521,2021-10,0.72,36.9,50.1,63.3,37.2,0.92,12.40,Yakima
1522,2021-11,1.65,33.0,42.1,51.2,34.3,0.46,6.39,Yakima


In [ ]:
#separate month and year
prism['year'] = prism['Date'].str[:4]
prism['month'] = prism['Date'].str[-2:]

#drop original date
prism.drop(columns='Date', inplace=True)

In [ ]:
p

,ppt (inches),tmin (degrees F),tmean (degrees F),tmax (degrees F),tdmean (degrees F),vpdmin (hPa),vpdmax (hPa),County,year,month
0,2.00,20.1,26.7,33.2,21.2,0.45,1.93,Adams,1895,01
1,0.38,28.4,36.3,44.2,25.7,0.78,4.76,Adams,1895,02
2,0.38,28.5,40.0,51.5,26.4,0.90,7.99,Adams,1895,03
3,0.41,36.3,50.8,65.3,31.3,1.68,15.49,Adams,1895,04
4,0.91,41.6,55.3,69.0,37.0,1.78,17.02,Adams,1895,05
...,...,...,...,...,...,...,...,...,...,...
1519,0.02,56.6,71.8,87.1,49.2,4.61,33.66,Yakima,2021,08
1520,0.52,46.7,61.6,76.5,43.7,2.04,22.51,Yakima,2021,09
1521,0.72,36.9,50.1,63.3,37.2,0.92,12.40,Yakima,2021,10
1522,1.65,33.0,42.1,51.2,34.3,0.46,6.39,Yakima,2021,11


In [ ]:
#download to github
from google.colab import drive
drive.mount('drive')

#avg.to_csv('average_temp.csv')
#!cp average_temp.csv "drive/My Drive/"

prism.to_csv('prism.csv')
!cp prism.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
